## Loading Modules

In [1]:
import os 
# import pickle
import numpy as np
from tqdm import tqdm

from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical,plot_model
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense,add,Dropout

2024-03-12 21:29:46.682963: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-12 21:29:46.743455: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-12 21:29:46.958250: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-12 21:29:47.814853: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
# import shutil

# # Path to the text file containing image names
# file_path = r'/home/marocos/thesis/med-image-captioning/data/captions/random_sample_20.txt'

# # Read the file and extract image names
# with open(file_path, 'r') as file:
#     lines = file.readlines()
# image_names = [line.split(',')[0] for line in lines]

# # Define the source and destination folder paths
# source_folder_path = r'/home/marocos/mic-images/'
# destination_folder_path = r'/home/marocos/thesis/med-image-captioning/images/'

# # Ensure the destination folder exists
# os.makedirs(destination_folder_path, exist_ok=True)

# # Iterate over the list of image names
# for image_name in image_names:
#     source_image_path = os.path.join(source_folder_path, image_name)
    
#     # Check if the image exists in the source folder
#     if os.path.exists(source_image_path):
#         destination_image_path = os.path.join(destination_folder_path, image_name)
        
#         # Copy the image to the destination folder
#         shutil.copy2(source_image_path, destination_image_path)
#         print(f"Copied {image_name} to destination folder.")
#     else:
#         print(f"Image {image_name} not found in source folder.")


Copied 8568d3eee073bcc03a4e5b5d79b9b4a359ed83a0_3-Figure2-1.png to destination folder.
Copied 42312d108fe75d681a3a26bb29148cb195126a87_4-Figure58-1.png to destination folder.
Copied 42ffe66e9a3a3eb53aca25fa193d6b1829de3884_2-Figure1-1.png to destination folder.
Copied 9ac485f9982cb70005d8f2c6d2e51d7f5ab2fa69_4-Figure2-1.png to destination folder.
Copied c3c50f87bd4f7d316868a0da9b5a31d7f7783afe_2-Figure2-1.png to destination folder.
Copied d8e9aaa8ec05a3ffe6a9b493d8a1ca061bf49e82_2-Figure2-1.png to destination folder.
Copied 9ab54a9c9b9427f18bd886b44588fa39e479e64d_3-Figure1-1.png to destination folder.
Copied 3926ffeb2f292e5e0bd2d12436de309c87c77524_2-Figure1-1.png to destination folder.
Copied 7d516defb1c2142e80faa07f26ee0f855c7fa87c_3-Figure2-1.png to destination folder.
Copied a2a2e0fa06804866e30097d11959b9095d37ad86_2-Figure2-1.png to destination folder.
Copied 4a71c423baf37ed7a4b651e46e3c5570e38c4bfc_2-Figure2-1.png to destination folder.
Copied 2a44e43a07f8ba2a3d64722696b7985e672

## Image Feature Extraction

In [2]:
model = VGG16()
# restarcture the model
model = Model(inputs=model.inputs,outputs=model.layers[-2].output)

In [3]:
print(model.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,260,544 (512.16 MB)

 Trainable params: 134,260,544 (512.16 MB)

 Non-trainable params: 0 (0.00 B)

None


In [4]:
## extract features from each photo in the directory
features={}
directory=r'/home/marocos/thesis/med-image-captioning/images'

for img_name in tqdm(os.listdir(directory)):
    filename = directory+'/'+img_name

    # load an image from file
    image = load_img(filename,target_size=(224,224))

    # convert the image pixels to a numpy array
    image = img_to_array(image)

    # reshape data for the model
    image = image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))

    # prepare the image for the VGG model
    image = preprocess_input(image)

    # get features
    feature = model.predict(image,verbose=0)

    # get image id
    image_id = img_name.split('.')[0]

    #save the features
    features[image_id] = feature

100%|██████████| 100/100 [00:16<00:00,  6.21it/s]


In [10]:
# # save to file
# pickle.dump(features,open(r'D:\med-image-captioning\data\image_features\features.pkl','wb'))

## Loading the captions

In [6]:
# load the caption features using pickle
with open(r'/home/marocos/thesis/med-image-captioning/data/captions/random_sample_20.txt','r') as file:
    captions_doc=file.read()

In [7]:
# create mapping of image captions
mapping={}

for line in tqdm(captions_doc.split('\n')):
    # split the line by comma
    tokens=line.split(',')

    if len(tokens) >= 2:
        image_id,caption=tokens[0],tokens[1]

        #remove filename from image_id
        image_id=image_id.split('.')[0]


        if image_id not in mapping:
            mapping[image_id]=[]

        mapping[image_id].append(caption)




100%|██████████| 101/101 [00:00<00:00, 379252.20it/s]


In [8]:
print(len(mapping))

100


In [9]:
## add start and end sequence to the captions
for key,captions in mapping.items():
    for i in range(len(captions)):
        captions[i]='<start> '+captions[i]+' <end>'

In [10]:
print(captions[0])

<start> chest mriscan showing cyst to the left of the esophagus overlying the vertebral bodies thoracoscopic view of esophageal duplication cysts <end>


In [11]:
print(mapping['0a60e80445d6be21f3da582eeeec7bec83d82e74_4-Figure1-1'])

['<start> clinical images doppler ultrasound and ultrasound of the uterine lower segment <end>']


In [12]:
all_captions=[]
for key in mapping.keys():
    [all_captions.append(caption) for caption in mapping[key]]

In [13]:
len(all_captions)

100

In [14]:
print(all_captions[:5])

['<start> computed tomography scans obtained and before and and and after combination anti egfr antibody therapy in case <end>', '<start> multispectral optoacoustic tomography msot imageof thevascularplexus ina test subject takenwith detector msot invision courtesy of itheramedical gmbh <end>', '<start> endoscopy picture pre treatment <end>', '<start> case lateral radiography and computed tomography scanning show an inadequate reposition of dislocated fracture with halo vest <end>', '<start> tl weighted images of abdominal magnetic resonance images show large masses as well as weighted images axial image sagittal image <end>']


In [15]:
# tokenize the text

tokenizer=Tokenizer()
tokenizer.fit_on_texts(all_captions)
vocab_size=len(tokenizer.word_index)+1

In [16]:
print(vocab_size)

620


In [17]:
max_length=max(len(caption.split()) for caption in all_captions)
print(max_length)

22


## Train test split

In [18]:
image_ids=list(mapping.keys())
split=int(len(image_ids)*0.8)
train=image_ids[:split]
test=image_ids[split:]


In [19]:
# create data generator
def data_generator(data_keys,mapping,features,tokenizer,max_length,vocab_size,batch_size):
    X1,X2,y= list(),list(),list()
    n=0
    while 1:
        for key in data_keys:
            n+=1
            captions=mapping[key]

            for caption in captions:
                sequence=tokenizer.texts_to_sequences([caption])[0]
                
                for i in range(1,len(sequence)):
                    # split into input and output pair
                    in_seq,out_seq=sequence[:i],sequence[i]

                    # pad input sequence
                    in_seq=pad_sequences([in_seq],maxlen=max_length)[0]

                    # encode output sequence
                    out_seq=to_categorical([out_seq],num_classes=vocab_size)[0]

                    # store the input and output sequence
                    X1.append(features[key][0])
                    X2.append(in_seq)
                    y.append(out_seq)
            if n==batch_size:
                X1,X2,y=np.array(X1),np.array(X2),np.array(y)
                yield [X1,X2],y
                X1,X2,y=list(),list(),list()
                n=0





## Model Building

In [20]:
# encoder
# image feature model
inputs1=Input(shape=(4096,))
fe1=Dropout(0.4)(inputs1)
fe2=Dense(256,activation='relu')(fe1)

# sequence feature layer
inputs2=Input(shape=(max_length,))
se1=Embedding(vocab_size,256,mask_zero=True)(inputs2)
se2=Dropout(0.4)(se1)
se3=LSTM(256)(se2)

# decoder
decoder1=add([fe2,se3])
decoder2=Dense(256,activation='relu')(decoder1)
outputs=Dense(vocab_size,activation='softmax')(decoder2)

# model definiton
model=Model(inputs=[inputs1,inputs2],outputs=outputs)
model.compile(loss='categorical_crossentropy',optimizer='adam')



## Train the model

In [27]:
epochs = 10
batch_size = 5
steps = len(train)//batch_size

for i in tqdm(range(epochs)):
    generator=data_generator(train,mapping,features,tokenizer,max_length,vocab_size,batch_size)
    
    model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)


  0%|          | 0/10 [00:00<?, ?it/s]


TypeError: `output_signature` must contain objects that are subclass of `tf.TypeSpec` but found <class 'list'> which is not.

In [ ]:
# model.save(r'D:\med-image-captioning\model\model_50.h5')

## Generate captions

In [ ]:
def index_to_word(integer,tokenizer):
    for word,index in tokenizer.word_index.items():
        if index==integer:
            return word
    return None

In [ ]:
# generate a description for an image
def predict_caption(model,image,tokenizer,max_length):
    in_text="<start>"

    for i in range(max_length):
        sequence=tokenizer.texts_to_sequences([in_text])[0]
        sequence=pad_sequences([sequence],max_length)

        yhat=model.predict([image,sequence],verbose=0)

        # convert probability to integer
        yhat=np.argmax(yhat)

        # convert index to word
        word=index_to_word(yhat,tokenizer)

        if word is None:
            break
        ## append as input for generating the next word
        in_text+=" "+word

        # stop if we cannot generate the next word
        if word=='<end>':
            break
    return in_text

In [ ]:
# evaluate the model

actual,predicted=list(),list()

for key in test:
    try:
        # get actual caption
        captions=mapping[key]

        # generate caption
        y_pred=predict_caption(model,features[key],tokenizer,max_length)

        #split into words
        actual_captions=[caption.split() for caption in captions]
        y_pred=y_pred.split()
        #append to the list
        actual.append(actual_captions)
        predicted.append(y_pred)
    except KeyError:
        continue

# calculate BLEU score




In [ ]:
from nltk.translate.bleu_score import corpus_bleu
# calculate BLEU score
print("BLUE-1: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
print("BLUE-2: %f" % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))

## Visualize the results

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
def generate_caption(image_name):

    # image_name="780d51420ad492a61d00295adb6919bd492e6bd3_8-Figure3-1.png"
    img_path=r'D:\med-image-captioning\images\\'+image_name
    img_name=image_name.split('.')[0]
    image=Image.open(img_path)
    captions=mapping[img_name]

    print("----------------Actual Captions----------------")
    print(captions)
    y_pred=predict_caption(model,features[img_name],tokenizer,max_length)
    print("----------------Predicted Caption----------------")
    print(y_pred)
    plt.imshow(image)


In [ ]:
generate_caption('0a60e80445d6be21f3da582eeeec7bec83d82e74_4-Figure1-1.png')